In [1]:
# Необходимые импорты
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import time
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
from typing import List

In [2]:
# сбор всей необходимой информации с карточки товара
def get_info_for_card(url: str):
        br = webdriver.Chrome()
        
        br.get(url)
        time.sleep(5)
        br.execute_script("window.scrollTo(0, 1280)")    # пролистываем страницу, чтобы вся необходимая информация прогрузилась (подгрузка идет динамически)
        time.sleep(5)
        page_add = br.page_source
        tree = BeautifulSoup(page_add, 'html')
        
        # сбор описания
        try:
            submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
        except:
            submit = br.find_element(By.XPATH, '/html/body/div[1]/div/div/button[1]')  # если выскочила плашка 18+
            submit.click()
            time.sleep(3)
            submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
        try:
            submit.click()
        except:
            br.execute_script("window.scrollTo(0, 1920)")    # пролистываем страницу ниже, чтобы пролистать реклату
            time.sleep(5)

            page_add = br.page_source
            tree = BeautifulSoup(page_add, 'html')

            submit.click()
            
        time.sleep(5)
        page_add = br.page_source
        tree_description = BeautifulSoup(page_add, 'html')
    
        br.close()
        
        image_card = tree.find_all('div', {'class': 'slide__content img-plug'})
        image_card = image_card[0].img['src']


        image_feedbacks = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
        image_feedbacks = None if len(image_feedbacks)==0 else image_feedbacks[0].img['src']
        
        description = tree_description.find_all('p', {'class': 'option__text'})
        
        return image_card, image_feedbacks, description


# скачивание картинки
def load_image(url, path, name):
    img = requests.get(url)
    full_path = path / name
    
    with open(full_path, "wb") as out:
        out.write(img.content)

In [3]:
def parsing_items_wb(
    urls: List[str],
    folders_name: List[str],
    root_dir: Path,        
    start_category: int = 0,
    start_item: int = 0):
#     try:
        for i in range(start_category, len(urls)):
            main_url = urls[i]
            br = webdriver.Chrome()
            br.get(main_url)
            time.sleep(5)
            page_add = br.page_source
            tree = BeautifulSoup(page_add, 'html')
            br.close()
            
            # берем только 20 первых товаров
            items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})[:20]
            items_href = [items[i]['href'] for i in range(len(items))]
            descriptions = {col: [] for col in ["item", "description"]}
            
            for j in tqdm(range(start_item, len(items_href))):
#                 print(items_href[j])
                image_card, image_feedbacks, description = get_info_for_card(items_href[j])
                
                path_card = root_dir / folders_name[i] / "card"
                path_feedbacks = root_dir / folders_name[i] / "feedbacks"
                path_description = root_dir / folders_name[i]
                
                if j==0:
                    path_card.mkdir(parents=True, exist_ok=True)
                    path_feedbacks.mkdir(parents=True, exist_ok=True)
                
                load_image(url=image_card, path=path_card, name=f"{j}.png")
                if image_feedbacks is not None:
                    load_image(url=image_feedbacks, path=path_feedbacks, name=f"{j}_feedbacks.png")
                
                descriptions["item"].append(f"{j}.png")
                descriptions["description"].append(None if len(description)==0 else description[0].text)
            
            pd.DataFrame(descriptions).to_csv(path_description / "descriptions.csv", index=False)
            print(f'Прошла категория "{Path(main_url).name}"')
            
#     except Exception as e:
#         print(e)        

In [4]:
categories = pd.read_csv("wb_categories.csv")
#categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
2,Женщинам,Верхняя одежда,NaN,https://www.wildberries.ru/catalog/zhenshchina...
3,Женщинам,"Джемперы, водолазки и кардиганы",NaN,https://www.wildberries.ru/catalog/zhenshchina...
4,Женщинам,Джинсы,NaN,https://www.wildberries.ru/catalog/zhenshchina...
...,...,...,...,...
1442,Канцтовары,Рисование и лепка,NaN,https://www.wildberries.ru/catalog/knigi-i-kan...
1443,Канцтовары,Счетный материал,NaN,https://www.wildberries.ru/catalog/knigi-i-dis...
1444,Канцтовары,Торговые принадлежности,NaN,https://www.wildberries.ru/catalog/kantstovary...
1445,Канцтовары,Чертежные принадлежности,NaN,https://www.wildberries.ru/catalog/knigi-i-kan...


In [5]:
folders_name = []

for i in range(len(categories)):
    if str(categories.iloc[i]["category_3"]) == "nan":
        if str(categories.iloc[i]["category_2"]) == "nan":
            cat = list(categories.iloc[i][:1])
        else:
            cat = list(categories.iloc[i][:2])
    else:
        cat = list(categories.iloc[i][:3])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder.replace(":", ""))

In [6]:
root_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")

In [7]:
# не смогло спарситься из-за другой структуры (мб баг)
folders_name[209], folders_name[217], folders_name[585], folders_name[588], folders_name[590], folders_name[593],\
folders_name[615]

('Детям_Детская&электроника_Игровые&консоли&и&игры',
 'Детям_Детская&электроника_Смартфоны&и&телефоны')

In [12]:
# не смогла спарситься из за 4ой вложенности категорий
folders_name[324], folders_name[327], folders_name[328], folders_name[343], folders_name[350], folders_name[358],\
folders_name[377], folders_name[514], folders_name[515], folders_name[662], folders_name[776], folders_name[948]

('Дом_Кухня_Кружки',
 'Дом_Кухня_Кухонный&текстиль',
 'Дом_Кухня_Магниты',
 'Дом_Предметы&интерьера_Картины&и&постеры',
 'Дом_Предметы&интерьера_Фоторамки&и&фотоальбомы',
 'Дом_Спальня_Постельные&принадлежности',
 'Дом_Досуг&и&творчество_Творчество&и&рукоделие',
 'Красота_Парфюмерия_Женские&ароматы',
 'Красота_Парфюмерия_Мужские&ароматы',
 'Электроника_Автоэлектроника&и&навигация_Запчасти&на&легковые&автомобили',
 'Электроника_Автоэлектроника&и&навигация_Краски&и&грунтовки',
 'Электроника_Автоэлектроника&и&навигация_Аксессуары&в&салон&и&багажник',
 'Электроника_Автоэлектроника&и&навигация_Другие&аксессуары&и&доп.&оборудование',
 'Электроника_Гарнитуры&и&наушники_Чехлы',
 'Электроника_Смартфоны&и&телефоны_Чехлы',
 'Мебель_Мебель&для&спальни_Матрасы',
 'Автотовары_Запчасти&на&легковые&автомобили_Ходовая&часть')

повтор категорий
от sim-karty до flash-nakopiteli (от 647 до 664)

Пропало описание с "Женщинам_Подарки&женщинам_Открытки&и&сувениры" включительно [111]

Начало описания с "Автотовары_Автокосметика&и&автохимия_Прочее&автохимия" [971]

In [ ]:
parsing_items_wb(urls=categories.url[:],
                 folders_name=folders_name,
                 root_dir=root_dir,
                 start_category=0,
                 start_item=0)

____

In [4]:
categories_4 = pd.read_csv("wb_categories_4.csv")
categories_4

,category_1,category_2,category_3,category_4,url
0,Дом,Кухня,Кружки,Для глинтвейна,https://www.wildberries.ru/catalog/kuhnya/posu...
1,Дом,Кухня,Кружки,Для детей,https://www.wildberries.ru/catalog/kuhnya/posu...
2,Дом,Кухня,Кружки,Для кофе,https://www.wildberries.ru/catalog/kuhnya/posu...
3,Дом,Кухня,Кружки,Для чая,https://www.wildberries.ru/catalog/kuhnya/posu...
4,Дом,Кухня,Кружки,Заварочные,https://www.wildberries.ru/catalog/kuhnya/posu...
...,...,...,...,...,...
145,Автотовары,Запчасти на легковые автомобили,Ходовая часть,Амортизаторы и комплектующие,https://www.wildberries.ru/catalog/avtotovary/...
146,Автотовары,Запчасти на легковые автомобили,Ходовая часть,Рычаги подвески,https://www.wildberries.ru/catalog/avtotovary/...
147,Автотовары,Запчасти на легковые автомобили,Ходовая часть,Ступицы и подшипники,https://www.wildberries.ru/catalog/avtotovary/...
148,Автотовары,Запчасти на легковые автомобили,Ходовая часть,Стабилизаторы и комплектующие,https://www.wildberries.ru/catalog/avtotovary/...


In [5]:
folders_name = []

for i in range(len(categories_4)):
    cat = list(categories_4.iloc[i][:4])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder.replace(":", ""))

In [7]:
root_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data_4")

In [18]:
# не смогло спарситься из-за другой структуры (мб баг)
folders_name[64]

'Дом_Спальня_Постельные&принадлежности_Матрасы'

In [19]:
# не смогла спарситься из за 5ой вложенности категорий
folders_name[49], folders_name[53], folders_name[76], folders_name[127], folders_name[128], folders_name[129]

('Дом_Предметы&интерьера_Картины&и&постеры_Картины',
 'Дом_Предметы&интерьера_Картины&и&постеры_Постеры',
 'Дом_Досуг&и&творчество_Творчество&и&рукоделие_Аксессуары&и&материалы&для&рукоделия',
 'Электроника_Смартфоны&и&телефоны_Чехлы_Чехлы&для&телефонов&Apple',
 'Электроника_Смартфоны&и&телефоны_Чехлы_Чехлы&для&телефонов&Xiaomi',
 'Электроника_Смартфоны&и&телефоны_Чехлы_Чехлы&для&телефонов&Samsung')

In [ ]:
parsing_items_wb(urls=categories_4.url[:],
                 folders_name=folders_name,
                 root_dir=root_dir,
                 start_category=0,
                 start_item=0)

___

In [4]:
categories_5 = pd.read_csv("wb_categories_5.csv")
categories_5

,category_1,category_2,category_3,category_4,category_5,url
0,Дом,Предметы интерьера,Картины и постеры,Картины,Арт и абстракция,https://www.wildberries.ru/catalog/dlya-doma/p...
1,Дом,Предметы интерьера,Картины и постеры,Картины,Архитектура,https://www.wildberries.ru/catalog/dlya-doma/p...
2,Дом,Предметы интерьера,Картины и постеры,Картины,Аниме,https://www.wildberries.ru/catalog/dlya-doma/p...
3,Дом,Предметы интерьера,Картины и постеры,Картины,Города и страны,https://www.wildberries.ru/catalog/dlya-doma/p...
4,Дом,Предметы интерьера,Картины и постеры,Картины,Детские,https://www.wildberries.ru/catalog/dlya-doma/p...
...,...,...,...,...,...,...
92,Электроника,Смартфоны и телефоны,Чехлы,Чехлы для телефонов Samsung,Серия Galaxy Ax4,https://www.wildberries.ru/catalog/elektronika...
93,Электроника,Смартфоны и телефоны,Чехлы,Чехлы для телефонов Samsung,Серия Galaxy S,https://www.wildberries.ru/catalog/elektronika...
94,Электроника,Смартфоны и телефоны,Чехлы,Чехлы для телефонов Samsung,Серия Galaxy Z Flip,https://www.wildberries.ru/catalog/elektronika...
95,Электроника,Смартфоны и телефоны,Чехлы,Чехлы для телефонов Samsung,Серия Galaxy Z Fold,https://www.wildberries.ru/catalog/elektronika...


In [5]:
folders_name = []

for i in range(len(categories_5)):
    cat = list(categories_5.iloc[i][:5])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder.replace(":", ""))

In [7]:
root_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data_5")

In [ ]:
parsing_items_wb(urls=categories_5.url[:],
                 folders_name=folders_name,
                 root_dir=root_dir,
                 start_category=0,
                 start_item=0)

___

# Тестирование

In [38]:
url = 'https://www.wildberries.ru/catalog/74299675/detail.aspx'
url = 'https://www.wildberries.ru/catalog/149410773/detail.aspx'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
br.execute_script("window.scrollTo(0, 1200)")
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [13]:
#tree

In [10]:
data = tree.find_all('div', {'class': 'slide__content img-plug'})
data[0].img['src']

'https://basket-10.wbbasket.ru/vol1494/part149410/149410773/images/big/1.webp'

In [41]:
data = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
data[0].img['src']

'https://feedback05.wbbasket.ru/vol1744/part174414/174414618/photos/ms.webp'

___

In [26]:
url = 'https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [33]:
data = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
print(len(data))
data[0]['href']

30


'https://www.wildberries.ru/catalog/160832044/detail.aspx'

In [39]:
items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
items_href = [items[i]['href'] for i in range(len(items))][:15]

___

In [17]:
categories = pd.read_csv("wb_categories.csv")
categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...


In [24]:
Path(categories.iloc[0].url).name

'bluzki-i-rubashki'

In [25]:
categories["url"]

0    https://www.wildberries.ru/catalog/zhenshchina...
1    https://www.wildberries.ru/catalog/zhenshchina...
Name: url, dtype: object

___

In [10]:
# # скачивание картинки
# img = "https://basket-05.wbbasket.ru/vol742/part74299/74299675/images/c246x328/1.webp"

# p = requests.get(img)

# with open("data/img1.png", "wb") as out:
#     out.write(p.content)

In [ ]:
data_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")

____

In [12]:
url = 'https://www.wildberries.ru/catalog/118183375/detail.aspx'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
br.execute_script("window.scrollTo(0, 1280)")
time.sleep(5)
submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
submit.click()
time.sleep(2)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [13]:
# submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
# submit

In [19]:
# submit.click()

In [20]:
# page_add = br.page_source
# tree = BeautifulSoup(page_add, 'html')

In [13]:
data = tree.find_all('p', {'class': 'option__text'})
data[0].text

'Красота, изящество, стиль и удобство\nКачественный пошив.Российское производство.\nЭлегантный вырез.Бесшовный материал.\n3-х ступенчатая застежка.\nДелает талию изящной.\n\nСтильное боди из коллекции российского бренда Delicieuse - элегантная и эффектная модель для модного женственного образа.\nБоди для танцев отличает длинный рукав, округлый вырез горловины, соблазнительно привлекающий внимание к области декольте сексуальный вырез на груди.\nВысокая посадка и открытый фасон подчеркивают изгиб бедер и делают талию изящной, эффектно выглядят в сочетании с джинсами, брюками или юбками с низкой посадкой.\n\nУ изделия трехступенчатая застежка - удлинитель с прочными крючками для удобства ношения, застегивания и расстегивания при надевании и снятии. \n\nБесшовную молодежную модель «в обтяжку» отличает качественный пошив и контроль всех этапов производства, материал премиум качества.\n\nМягко обтягивающее боди в лаконичном декоре дополнит гардероб взрослых женщин и девушек - подростков, под

___

In [17]:
url = 'https://www.wildberries.ru/catalog/146854891/detail.aspx'
url = 'https://www.wildberries.ru/catalog/118183375/detail.aspx'

br = webdriver.Chrome()
br.set_window_size(1920, 1080)
br.get(url)
#br.fullscreen_window()
time.sleep(5)
br.execute_script("window.scrollTo(0, 1280)")
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')
try:
    submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
except:
    submit = br.find_element(By.XPATH, '/html/body/div[1]/div/div/button[1]')
    submit.click()
    submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
submit.click()
time.sleep(2)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [22]:
data = tree.find_all('p', {'class': 'option__text'})
data[0].text

'Подарочная открытка или купон на желание отличный парный подарок для двоих влюбленных. Сертификат поздравление внесет разнообразие в жизнь партнеров. Абонемент можно использовать, как сувенир, дополнение к основному подарку для пар, на свадьбу молодоженам или годовщину, юбилей свадьбы любимому мужчине или женщине. Романтический сертификат в качестве мужского подарка подарит незабываемые эмоции парню или другу на день рождения, 14 февраля или 23 февраля. Подарите смешную прикол открытку в качестве валентинки на день всех влюбленных мужу или жене, как проявление любви. Абонемент креативный и необычный подарок признание на др, 8 марта, нг, новый год. Удивите подругу, девушку мини записками из серии про love. Купон с сердечками со стирающимся скретч слоем можно использовать как оригинальный сюрприз для него и для нее на новогодний праздник или на год отношений. Сертификат упакован в непрозрачный плотный картонный конверт, обеспечивая сохранность при транспортировке и конфиденциальность со

___

In [14]:
url = 'https://www.wildberries.ru/catalog/17765673/detail.aspx'

image_card, image_feedbacks, description = get_info_for_card(url)

In [101]:
        url = 'https://www.wildberries.ru/catalog/118183375/detail.aspx'
        br = webdriver.Chrome()
#         br.set_window_size(1280, 1800)
        br.get(url)
        time.sleep(5)
        br.execute_script("window.scrollTo(0, 1280)")    # пролистываем страницу, чтобы вся необходимая информация прогрузилась (подгрузка идет динамически)
        time.sleep(5)
#         br.execute_script("window.scrollTo(0, 1920)")    # пролистываем страницу, чтобы вся необходимая информация прогрузилась (подгрузка идет динамически)
#         time.sleep(5)
        page_add = br.page_source
        tree = BeautifulSoup(page_add, 'html')
        
        # сбор описания
        try:
            submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
#             submit.location_once_scrolled_into_view
#             br.execute_script('arguments[0].scrollIntoView()', submit)
#             time.sleep(5)
        except:
            submit = br.find_element(By.XPATH, '/html/body/div[1]/div/div/button[1]')  # если выскочила плашка 18+
            submit.click()
            time.sleep(3)
            submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
        try:
            submit.click()
        except:
#             br.execute_script("window.scrollTo(0, 1920)")    # пролистываем страницу ниже, чтобы пролистать реклату
#             time.sleep(5)

#             page_add = br.page_source
#             tree = BeautifulSoup(page_add, 'html')

            submit.click()
            
        time.sleep(5)
        page_add = br.page_source
        tree_description = BeautifulSoup(page_add, 'html')
    
        br.close()
        
        image_card = tree.find_all('div', {'class': 'slide__content img-plug'})
        image_card = image_card[0].img['src']


        image_feedbacks = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
#         image_feedbacks = None if len(image_feedbacks)==0 else image_feedbacks[0].img['src']
        image_feedbacks = image_feedbacks[0].img['src']

        
        description = tree_description.find_all('p', {'class': 'option__text'})

In [102]:
image_card, image_feedbacks, description

('https://basket-09.wbbasket.ru/vol1181/part118183/118183375/images/big/1.webp',
 'https://feedback05.wbbasket.ru/vol1744/part174452/174452993/photos/ms.webp',
 [<p class="option__text">Наш инновационный фиксатор резьбы разработан с использованием передовых технологий, обеспечивая непревзойденную фиксацию даже в условиях высоких нагрузок и вибрации. 
  В бытовых условиях фиксатор резьбы также находит свое применение. Вы можете использовать его для надежного закрепления мебельных соединений. Например, в сборке столов, стульев, полок и шкафов. Он предотвратит ослабление соединений и обеспечит стабильность и безопасность использования мебели.
  Кроме того, фиксатор резьбы может быть полезен при ремонте бытовой техники. Он поможет закрепить резьбовые соединения внутри устройств, таких как компьютеры, телевизоры, кухонные приборы и другие бытовые электронные устройства. 
  
  Резьбовой фиксатор красный неразьемный и синий разъемный, является пломбировочной краской, применяемой на таких комп

In [1]:
#tree

In [2]:
#tree

In [3]:
# new_tree = tree.find_all('div', {'class': 'swiper-wrapper'})[0]
# new_tree

In [100]:
new_tree.find_all('div', {'class': 'swiper-slide img-plug'})

[<div class="swiper-slide img-plug" data-link="class{merge: isVideo toggle='user-photos__video-slide'}" style="margin-right: 8px;"> <img alt="photo" height="293" src="https://feedback05.wbbasket.ru/vol1640/part164096/164096636/photos/ms.webp" width="220"/> </div>,
 <div class="swiper-slide img-plug" data-link="class{merge: isVideo toggle='user-photos__video-slide'}" style="margin-right: 8px;"> <img alt="photo" height="293" src="https://feedback05.wbbasket.ru/vol1640/part164096/164096637/photos/ms.webp" width="220"/> </div>,
 <div class="swiper-slide img-plug" data-link="class{merge: isVideo toggle='user-photos__video-slide'}" style="margin-right: 8px;"> <img alt="photo" height="293" src="https://feedback05.wbbasket.ru/vol1640/part164094/164094844/photos/ms.webp" width="220"/> </div>,
 <div class="swiper-slide img-plug" data-link="class{merge: isVideo toggle='user-photos__video-slide'}" style="margin-right: 8px;"> <img alt="photo" height="293" src="https://feedback05.wbbasket.ru/vol1640

In [ ]:
tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})

"user-photos"